# Задание №3

Все встроенные математические функции языка Julia поддерживают векторные действия с массивами, то есть если им в качестве аргумента передать не скалярное значение, а массив, то функция будет вычислена от каждого элемента массива. Для этого после названия функции нужно указать точку. Например, если мы хотим вычислить синус от каждого элемента массива A, то должны вызвать функцию так: sin.(A). 

Потенциально, такие функции могут использовать SIMD регистры процессора, что должно давать выигрыш в производительности по сравнению с вычислениями в цикле. 

В качестве третьего задания придумайте некоторую свою функцию, которая манипулирует числовым массивом и вычисляет какое-то значения. Составьте ее из элементарных математических функций и вычислите ее значение от массива двумя способами.


- Первый способ заключается в вычислении значений функций от элементов массива в цикле, передавая каждый элемент массива в функцию по отдельности.
- Второй способ заключается в передаче всего массива в виде аргумента.

Второй способ теоретически должен дать выигрыш в производительности. Замерьте время выполнения с помощью @time.

Прочитайте в официальной документации о макросах @inbounds, @fastmath и @simd и примените их в своем цикле. Обязательно проверьте корректность вычислений при использовании @fastmath (как это сделать?)


In [1]:
# Custom function to calculate the square of sine for each element
function square_sin(arr)
    result = similar(arr) # Create an empty array of the same size to store results
    for i in eachindex(arr)
        result[i] = sin(arr[i])^2
    end
    return result
end


square_sin (generic function with 1 method)

In [2]:
# Method 1: Calculate values in a loop
function method1(arr)
    result = similar(arr)
    for i in eachindex(arr)
        result[i] = sin(arr[i])^2
    end
    return result
end

# Test Method 1
arr = rand(1_000_000) # Create a large random array for testing
@time result1 = method1(arr)


  0.105916 seconds (270.75 k allocations: 21.098 MiB, 10.10% gc time, 82.48% compilation time)


1000000-element Vector{Float64}:
 0.02884200530483517
 0.5355422450382689
 0.5347487938807369
 0.13485998950521652
 0.07380951688291382
 0.3197493695571464
 0.6664327772758084
 0.013546783221146486
 0.6887345316258968
 0.07229809731701921
 ⋮
 0.3402069176781026
 0.0449101829021944
 0.42448895263084246
 0.30030975752474964
 0.28059818919713825
 0.36180474977342125
 0.3316432059840624
 0.2539386146242875
 0.6129988502311188

In [3]:
# Method 2: Pass the entire array as an argument
function method2(arr)
    return sin.(arr).^2
end

# Test Method 2
@time result2 = method2(arr)


  0.137707 seconds (270.99 k allocations: 22.162 MiB, 58.97% gc time, 33.90% compilation time)


1000000-element Vector{Float64}:
 0.02884200530483517
 0.5355422450382689
 0.5347487938807369
 0.13485998950521652
 0.07380951688291382
 0.3197493695571464
 0.6664327772758084
 0.013546783221146486
 0.6887345316258968
 0.07229809731701921
 ⋮
 0.3402069176781026
 0.0449101829021944
 0.42448895263084246
 0.30030975752474964
 0.28059818919713825
 0.36180474977342125
 0.3316432059840624
 0.2539386146242875
 0.6129988502311188

In [4]:
function method1_optimized(arr)
    result = similar(arr)
    @simd for i in eachindex(arr)
        @inbounds result[i] = @fastmath sin(arr[i])^2
    end
    return result
end

# Test optimized Method 1
@time result1_optimized = method1_optimized(arr)


  0.017276 seconds (17.47 k allocations: 8.565 MiB, 49.89% compilation time)


1000000-element Vector{Float64}:
 0.02884200530483517
 0.5355422450382689
 0.5347487938807369
 0.13485998950521652
 0.07380951688291382
 0.3197493695571464
 0.6664327772758084
 0.013546783221146486
 0.6887345316258968
 0.07229809731701921
 ⋮
 0.3402069176781026
 0.0449101829021944
 0.42448895263084246
 0.30030975752474964
 0.28059818919713825
 0.36180474977342125
 0.3316432059840624
 0.2539386146242875
 0.6129988502311188

# Задание №4
Julia обладает большой коллекцией встроенных функций, которые позволяют писать код в функциональном стиле. Написанный в таком стиле код очень хорошо поддается параллелизации, так как используемые функции
являются чистыми и не должны модифицировать свои аргументы, что предотвращает гонку данных.

- Изучите функцию reduce. Расскажите своими словами как она работает.
- Создайте небольшой массив целых чисел, такой, чтобы можно было проверить корректность вычислений. С помощью reduce сделайте с ним следующие действия.
  - Найдите все положительные числа, отрицательные числа, четные, нечетные, делящиеся без остатка на 7. Затем с получившимися в результате такой фильтрации последовательностями проделайте следующие операции: просуммируйте, найдите максимум, минимум, среднее, выборочную дисперсию.
  - Попробуйте сразу создать массив с перечисленными выше условиями, то есть например такой, который состоит из целых чисел, делящихся без остатка на 7.
- Объедините все условия вместе, то есть например найдите сумму всех элементов, которые делятся на 7 без остатка, при этом положительные, не больше какого-то числа.
- Задание можно выполнить множеством способов. Защита лабораторной пройдет проще, если будут использованы анонимные функции, оператор |> и функции из модуля Base.Iter

In [1]:
using Statistics  # Import the Statistics module for mean function

arr = [1, 2, -3, 4, 5, -6, 7, 14, 21]

# Find positive numbers, negative numbers, even, odd, and divisible by 7
positives = arr |> x -> filter(y -> y > 0, x)
negatives = arr |> x -> filter(y -> y < 0, x)
even_numbers = arr |> x -> filter(y -> y % 2 == 0, x)
odd_numbers = arr |> x -> filter(y -> y % 2 != 0, x)
divisible_by_7 = arr |> x -> filter(y -> y % 7 == 0, x)

# Perform operations on filtered sequences
sum_positives = positives |> x -> reduce(+, x)
max_negatives = negatives |> x -> reduce(max, x)
min_even = even_numbers |> x -> reduce(min, x)
average_odd = odd_numbers |> x -> mean(x)  # Use mean function from Statistics module
variance_div7 = divisible_by_7 |> x -> sum((i - mean(x))^2 for i in x) / (length(x) - 1)

# Print the results
println("Положительные числа: $positives")
println("Отрицательные числа: $negatives")
println("Четные числа: $even_numbers")
println("Нечетные числа: $odd_numbers")
println("Сумма положительных чисел: $sum_positives")
println("Максимальное количество отрицательных чисел: $max_negatives")
println("Минимум четных чисел: $min_even")
println("Среднее значение нечетных чисел: $average_odd")
println("Sample variance of numbers divisible by 7: $variance_div7")

Положительные числа: [1, 2, 4, 5, 7, 14, 21]
Отрицательные числа: [-3, -6]
Четные числа: [2, 4, -6, 14]
Нечетные числа: [1, -3, 5, 7, 21]
Сумма положительных чисел: 54
Максимальное количество отрицательных чисел: -3
Минимум четных чисел: -6
Среднее значение нечетных чисел: 6.2
Sample variance of numbers divisible by 7: 49.0


In [2]:
divisible_by_7_directly = [x for x in arr if x % 7 == 0]


3-element Vector{Int64}:
  7
 14
 21

In [3]:
sum_divisible_by_7_positive_limit = arr |> x -> filter(y -> y > 0 && y % 7 == 0, x) |> x -> sum(x)


42